In [28]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import time

In [10]:
# Functions to generate y1 and y2 from X_train and X_test
def generate_y1(array):
    return 2 * np.sum(array) + np.random.randn()

def generate_y2(array):
    if np.sum(array) >= 0:
        return 1
    else:
        return 0

train_size = 8000
test_size = 2000

input_size = 20
hidden_sizes = [50, 50]

In [13]:
np.random.seed(0)

X_train = np.random.randn(train_size, input_size).astype(np.float32)
X_test = np.random.randn(test_size, input_size).astype(np.float32)
y1_train = np.apply_along_axis(func1d=generate_y1, axis=1, arr=X_train)
y1_test = np.apply_along_axis(func1d=generate_y1, axis=1, arr=X_test)
y2_train = np.apply_along_axis(func1d=generate_y2, axis=1, arr=X_train)
y2_test = np.apply_along_axis(func1d=generate_y2, axis=1, arr=X_test)

print('Shape of X_train:', X_train.shape)
print('Shape of X_test:', X_test.shape)
print('Shape of y1_train:', y1_train.shape)
print('Shape of y1_test:', y1_test.shape)
print('Shape of y2_train:', y2_train.shape)
print('Shape of y2_test:', y2_test.shape)

Shape of X_train: (8000, 20)
Shape of X_test: (2000, 20)
Shape of y1_train: (8000,)
Shape of y1_test: (2000,)
Shape of y2_train: (8000,)
Shape of y2_test: (2000,)


In [12]:
X_train

array([[ 1.7640524 ,  0.4001572 ,  0.978738  , ..., -0.20515826,
         0.3130677 , -0.85409576],
       [-2.5529897 ,  0.6536186 ,  0.8644362 , ...,  1.2023798 ,
        -0.3873268 , -0.30230275],
       [-1.048553  , -1.420018  , -1.7062702 , ...,  0.3024719 ,
        -0.6343221 , -0.36274117],
       ...,
       [-0.27115998,  0.41401708,  0.33879587, ..., -0.2900122 ,
         0.09167031, -0.49718222],
       [-0.12011925,  0.6960229 , -2.2862267 , ..., -0.1664335 ,
         2.6251683 ,  0.53107285],
       [ 0.35580438,  0.15503596,  0.10216598, ...,  0.35143372,
         0.5613195 , -1.3860302 ]], dtype=float32)

In [16]:
class MultiOutputDataset(Dataset):
    def __init__(self,X,y1,y2):
        self.X = X 
        self.y1 = y1 
        self.y2 = y2 
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        return self.X[idx], self.y1[idx], self.y2[idx]

train_loader = DataLoader(dataset=MultiOutputDataset(X_train, y1_train, y2_train), batch_size=8, shuffle=True)
test_loader = DataLoader(dataset=MultiOutputDataset(X_test, y1_test, y2_test), batch_size=8, shuffle=True)


In [19]:
class MultipleOutputModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(MultipleOutputModel, self).__init__()
        self.input_size = input_size
        self.fcs = []  # List of fully connected layers
        in_size = input_size
        
        for i, next_size in enumerate(hidden_sizes):
            fc = nn.Linear(in_features=in_size, out_features=next_size)
            in_size = next_size
            self.__setattr__('fc{}'.format(i), fc)  # # set name for each fullly connected layer
            self.fcs.append(fc)
            
        self.last_fc = nn.Linear(in_features=in_size, out_features=1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        for i, fc in enumerate(self.fcs):
            x = fc(x)
            x = nn.ReLU()(x)
        out1 = self.last_fc(x)
        x2 = self.last_fc(x)
        out2 = self.sigmoid(nn.ReLU()(x2))
        return out1, out2

In [27]:
# Set mean squared error loss for y1 and binary cross entropy loss for y2
# model = LinearRegression(input_size, output_size)

model_pytorch = MultipleOutputModel(input_size, hidden_sizes)

criterion1 = nn.MSELoss()
criterion2 = nn.BCELoss()
optimizer = optim.Adam(model_pytorch.parameters(), lr=1e-4)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
# Train model
num_epochs = 20
time_start = time.time()

for epoch in range(num_epochs):
    model_pytorch.train()

    train_loss_total = 0
    
    for data, target1, target2 in train_loader:
        data, target1, target2 = data.to(device), target1.float().to(device), target2.float().to(device)
        optimizer.zero_grad()
        output1, output2 = model_pytorch(data)
        train_loss_1 = criterion1(output1.squeeze(), target1)
        train_loss_2 = criterion2(output2.squeeze(), target2)
        train_loss = torch.add(train_loss_1, train_loss_2)
        train_loss.backward()
        optimizer.step()
        train_loss_total += train_loss.item() * data.size(0)
        
    print('Epoch {} completed. Train loss is {:.3f}'.format(epoch + 1, train_loss_total / train_size))
    print('Time taken to completed {} epochs: {:.2f} minutes'.format(num_epochs, (time.time() - time_start) / 60))

# Evaluate model
model_pytorch.eval()

test_loss_total = 0
total_num_corrects = 0
threshold = 0.9
time_start = time.time()

for data, target1, target2 in test_loader:
    data, target1, target2 = data.to(device), target1.float().to(device), target2.float().to(device)
    output1, output2 = model_pytorch(data)
    test_loss_1 = criterion1(output1.squeeze(), target1)
    test_loss_2 = criterion2(output2.squeeze(), target2)
    test_loss = torch.add(test_loss_1, test_loss_2)
    test_loss.backward()
    optimizer.step()
    test_loss_total += test_loss.item() * data.size(0)
    
    pred = (output2 >= threshold).view_as(target2)  # to make pred have same shape as target
    num_correct = torch.sum(pred == target2.byte()).item()
    total_num_corrects += num_correct

print('Evaluation completed. Test loss is {:.3f}'.format(test_loss_total / test_size))
print('Test accuracy is {:.3f}'.format(total_num_corrects / test_size))
print('Time taken to complete evaluation: {:.2f} minutes'.format((time.time() - time_start) / 60))

Epoch 1 completed. Train loss is 70.786
Time taken to completed 20 epochs: 0.02 minutes
Epoch 2 completed. Train loss is 15.507
Time taken to completed 20 epochs: 0.04 minutes
Epoch 3 completed. Train loss is 2.432
Time taken to completed 20 epochs: 0.05 minutes
Epoch 4 completed. Train loss is 1.952
Time taken to completed 20 epochs: 0.07 minutes
Epoch 5 completed. Train loss is 1.781
Time taken to completed 20 epochs: 0.09 minutes
Epoch 6 completed. Train loss is 1.692
Time taken to completed 20 epochs: 0.11 minutes
Epoch 7 completed. Train loss is 1.640
Time taken to completed 20 epochs: 0.12 minutes
Epoch 8 completed. Train loss is 1.604
Time taken to completed 20 epochs: 0.14 minutes
Epoch 9 completed. Train loss is 1.573
Time taken to completed 20 epochs: 0.16 minutes
Epoch 10 completed. Train loss is 1.552
Time taken to completed 20 epochs: 0.17 minutes
Epoch 11 completed. Train loss is 1.531
Time taken to completed 20 epochs: 0.19 minutes
Epoch 12 completed. Train loss is 1.516